In [1]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import random
import torch
import torch.nn as nn
from torch_geometric.loader import DataLoader
from InterleavedGCNN import InterleavedGCNN
from GraphDataset import GraphDataset
from train_val_test import train
from parentBCELoss import parentBCELoss

#### Parent ILP

In [2]:
# Load dataset
train_dir = f'./../dataset/training/parent_graphs'
train_data = GraphDataset(root=train_dir, data_list=[])
parents_loader = DataLoader(train_data, batch_size=50, shuffle=True)

/Users/yadongzhang/Documents/GitHub/ML4CO/src/GraphDataset.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = torch.load(self.processed_paths[0])


In [3]:
data = parents_loader.dataset[0]
data

HeteroData(
  name={ ILP='SC_1.mps' },
  var_nodes={
    x=[542, 3],
    x_sub_opt=[542],
    mask=[542],
    prob_divisor=[1],
    prob_dividend=[1],
    prob=[1],
  },
  constr_nodes={ x=[449, 7] },
  (var_nodes, in, constr_nodes)={
    edge_index=[2, 4413],
    edge_attr=[4413, 1],
  },
  (constr_nodes, rev_in, var_nodes)={
    edge_index=[2, 4413],
    edge_attr=[4413, 1],
  }
)

#### Children ILP

In [4]:
# Load dataset
train_dir = f'./../dataset/training/children_graphs'
train_data = GraphDataset(root=train_dir, data_list=[])
children_loader = DataLoader(train_data, batch_size=50, shuffle=True)

In [5]:
data = children_loader.dataset[0]
data

HeteroData(
  name='SC_10_children_ILP_0.15',
  var_nodes={
    x=[989, 3],
    y=[989],
    mask=[989],
  },
  constr_nodes={ x=[1378, 7] },
  (var_nodes, in, constr_nodes)={
    edge_index=[2, 13837],
    edge_attr=[13837, 1],
  },
  (constr_nodes, rev_in, var_nodes)={
    edge_index=[2, 13837],
    edge_attr=[13837, 1],
  }
)

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
hidden_channels = 64
num_layers = 3

In [7]:
model = InterleavedGCNN(
    var_in_channels=children_loader.dataset[0]["var_nodes"].x.shape[1],
    cons_in_channels=children_loader.dataset[0]["constr_nodes"].x.shape[1],
    hidden_channels=hidden_channels,
    num_layers=num_layers,
    edge_attr_dim=1
).to(device)

# Define loss function and optimizer
criterion = {"children": nn.BCELoss(),
                "parents": nn.BCELoss()}
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


## Training and validation
num_epochs = 5
train_loss = []
val_loss = []

In [8]:
for epoch in range(1, num_epochs + 1):
    losses = train(model = model, children_loader = children_loader, 
                    parents_loader = parents_loader, 
                    children_criterion = criterion["children"],
                    parents_criterion = criterion["parents"],
                    optimizer = optimizer, device = device)
    train_loss.append(losses[0])
    val_loss.append(losses[1])
    print(f"Epoch: {epoch:02d}, Train Loss: {losses[0]:.4f}, Val Loss: {losses[1]:.4f}")

tensor([0.7881, 0.4513, 0.3111,  ..., 0.2193, 0.4596, 0.5361],
       grad_fn=<SqueezeBackward1>)
tensor([0.3689, 0.4888, 0.1519,  ..., 0.5258, 0.3972, 0.6186],
       grad_fn=<SqueezeBackward1>)
tensor([0.5856, 0.3467, 0.1867,  ..., 0.4476, 0.4306, 0.1855],
       grad_fn=<SqueezeBackward1>)
tensor([0.5886, 0.1428, 0.3699,  ..., 0.1744, 0.2212, 0.1860],
       grad_fn=<SqueezeBackward1>)
tensor([0.3435, 0.2375, 0.1110,  ..., 0.4759, 0.0995, 0.0061],
       grad_fn=<SqueezeBackward1>)
tensor([0.0710, 0.4327, 0.2498,  ..., 0.1026, 0.2162, 0.1806],
       grad_fn=<SqueezeBackward1>)
tensor([0.0130, 0.1011, 0.2270,  ..., 0.0058, 0.0418, 0.2944],
       grad_fn=<SqueezeBackward1>)
tensor([0.4375, 0.3690, 0.3274,  ..., 0.0096, 0.0789, 0.0646],
       grad_fn=<SqueezeBackward1>)
tensor([1.4056e-01, 6.1114e-02, 1.5839e-04,  ..., 1.3109e-01, 2.2500e-01,
        2.1006e-01], grad_fn=<SqueezeBackward1>)
tensor([0.0502, 0.1242, 0.1808,  ..., 0.0409, 0.3565, 0.0035],
       grad_fn=<SqueezeBackwar

RuntimeError: all elements of target should be between 0 and 1

In [10]:
---> 69 loss = criterion(out[data["var_nodes"].mask],
     70                  data['var_nodes'].y[data["var_nodes"].mask])
RuntimeError: all elements of target should be between 0 and 1

tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.0000)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.0000)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.0000)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
